<a href="https://colab.research.google.com/github/jamieoliver/us-patents-240918/blob/main/us-patents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# US Patent Phrase to Phrase Matching

Based on https://www.kaggle.com/code/jhoward/getting-started-with-nlp-for-absolute-beginners.

## Plan
- Initial Setup
  - Install relevant libraries
- Prepare Datasets
  - Download
  - Tokenise
  - Split into:
    - Training
    - Validation
    - Test
- Train Model
- Test Model

## Initial Setup

In [1]:
!pip install -Uqq huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.4/436.4 kB 5.1 MB/s eta 0:00:00


## Prepare Datasets

In [2]:
from huggingface_hub import *

training_path = snapshot_download('jamieoliver/us-patents-240918',
                                  repo_type='dataset',
                                  allow_patterns='*.csv')

test_path = snapshot_download('jamieoliver/us-patents-test-240918',
                              repo_type='dataset',
                              allow_patterns='*.csv')

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

train.csv:   0%|          | 0.00/2.14M [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

test.csv:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

In [3]:
import pandas as pd
import os.path

training_data = pd.read_csv(os.path.join(training_path, 'train.csv'))
training_data

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [4]:
test_data = pd.read_csv(os.path.join(test_path, 'test.csv'))
test_data

,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04
5,474c874d0c07bd21,dry corn,dry corn starch,C12
6,442c114ed5c4e3c9,tunneling capacitor,capacitor housing,G11
7,b8ae62ea5e1d8bdb,angular contact bearing,contact therapy radiation,B23
8,faaddaf8fcba8a3f,produce liquid hydrocarbons,produce a treated stream,C10
9,ae0262c02566d2ce,diesel fuel tank,diesel fuel tanks,F02
